In [1]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from utils import *
from copy import copy

dataSetPath = '../../../dataset/semeval2017-task8/'
dim = 5000

with open(dataSetPath + 'trainSet.json', 'r') as f:
    content = f.read()
trainSet = json.loads(content)
threads = []
for threadId in trainSet['threadIds']:
    thread = []
    structure = trainSet['structures'][threadId]
    ids = flattenStructure(structure)
    time2Id = {}
    for id in ids:
        if id in trainSet['posts']:
            time2Id[str(trainSet['posts'][id]['time'])] = id
    # post按照时间先后排序
    time2Id = sorted(time2Id.items(), key=lambda d: d[0])
    for (time, id) in time2Id:
        if id in trainSet['posts']:
            thread.append(trainSet['posts'][id]['text'])
    threads.append(thread)

with open(dataSetPath + 'testSet.json', 'r') as f:
    content = f.read()
testSet = json.loads(content)

for threadId in testSet['threadIds']:
    thread = []
    structure = testSet['structures'][threadId]
    ids = flattenStructure(structure)
    time2Id = {}
    for id in ids:
        if id in testSet['posts']:
            time2Id[str(testSet['posts'][id]['time'])] = id
    # post按照时间先后排序
    time2Id = sorted(time2Id.items(), key=lambda d: d[0])
    for (time, id) in time2Id:
        if id in testSet['posts']:
            thread.append(testSet['posts'][id]['text'])
    threads.append(thread)

cropus = []
for thread in threads:
    for text in thread:
        cropus.append(text)
tfidf_vec = TfidfVectorizer(max_features=5000)
tfidf_vec.fit(cropus)
label2IndexRumor = trainSet['label2IndexRumor'] if 'label2IndexRumor' in trainSet else None
label2IndexStance = trainSet['label2IndexStance'] if 'label2IndexStance' in trainSet else None


In [2]:
print(len(tfidf_vec.vocabulary_))

5000


In [3]:
# with open(dataSetPath + 'testSet.json', 'r') as f:
#     content = f.read()
# testSet = json.loads(content)
totalSet = {}
for key in trainSet:
    if isinstance(trainSet[key], list):
        totalSet[key] = copy(trainSet[key]) 
        totalSet[key] += testSet[key]
    elif isinstance(trainSet[key], dict):
        totalSet[key] = copy(trainSet[key])
        totalSet[key].update(testSet[key])

In [4]:
with open('./semeval_id_label.txt', 'w') as f:
    for key in totalSet['rumorTag']:
        f.write('{:s} {:d}\n'.format(key, totalSet['label2IndexRumor'][totalSet['rumorTag'][key]]))

In [5]:
def structure2graph(wordlist, structure, posts, contents, id, curIndex: list, parent=None):
    if id in posts:
        post = posts[id]['text']
        word2feq = {}
        for word in post.split(' '):
            if word in word2feq:
                word2feq[word] += 1
            else:
                word2feq[word] = 1
        
        content = ''
        if parent == None:
            content += '{:s}\t{:s}\t{:d}\t'.format(id, 'None', curIndex[0])
        else:
            content += '{:s}\t{:d}\t{:d}\t'.format(id, parent, curIndex[0])
        vec = ''
        for word in word2feq:
            if word in wordlist :
                vec += '{:d}:{:d} '.format(wordlist[word], word2feq[word])
        vec = vec[0:-1] + '\n'
        if len(vec) == 1:
            print(post)
        content += vec
        contents.append(content)

    if not structure:
        return

    p = cur[0]
    for cid in structure:
        cur[0] += 1
        structure2graph(wordlist, structure[cid], posts, contents, cid, curIndex, p)
        
    return 

In [6]:
with open('./semevaltree.txt', 'w') as f:
    for tid in totalSet['threadIds']:
        struct = totalSet['structures'][tid]
        posts = totalSet['posts']
        contents = []
        cur = [1]
        structure2graph(tfidf_vec.vocabulary_, struct, posts, contents, tid, cur, parent=None)
        for content in contents:
            f.write(content)